In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bike-sharing-dataset/hour.csv
/kaggle/input/bike-sharing-dataset/Readme.txt
/kaggle/input/bike-sharing-dataset/day.csv
/kaggle/input/bike-sharing-dataset/bike-sharing-dataset/hour.csv
/kaggle/input/bike-sharing-dataset/bike-sharing-dataset/Readme.txt
/kaggle/input/bike-sharing-dataset/bike-sharing-dataset/day.csv


In [2]:
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
df = pd.read_csv("../input/bike-sharing-dataset/day.csv")

In [4]:
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [5]:
df.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

In [6]:
df.drop(['instant', 'dteday'], inplace=True ,axis=1)

In [7]:
correlations = df.corrwith(df['cnt'])
print(correlations)

season        0.406100
yr            0.566710
mnth          0.279977
holiday      -0.068348
weekday       0.067443
workingday    0.061156
weathersit   -0.297391
temp          0.627494
atemp         0.631066
hum          -0.100659
windspeed    -0.234545
casual        0.672804
registered    0.945517
cnt           1.000000
dtype: float64


Features with positive correlation are: 
* **season**
* **yr**: year
* **mnth**: mmonths
* **weekday**: days of the week
* **workingday**: whether the day is neither a weekend nor holiday
* **temp**: temperature in Celsius
* **atemp**: "feels like" temperature in Celsius
* **casual**: number of non-registered user rentals initiated
* **registered**: number of registered user rentals initiated

The feature with highest correlation is **'registered'**.

Features with negative correlation are: 
* **holiday**: whether the day is considered a holiday
* **weathersit**
* **hum**: humidity
* **windspeed**

In [8]:
df.drop(['casual', 'registered'], inplace=True ,axis=1)

In [9]:
df.head()


,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,985
1,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,801
2,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,1349
3,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,1562
4,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,1600


In [10]:
import category_encoders as ce

In [11]:
encoder = ce.OneHotEncoder(cols=('season','yr','mnth','holiday','weekday','workingday','weathersit'), use_cat_names = True)

In [12]:
from sklearn.model_selection import train_test_split


In [13]:
train, test = train_test_split(df, random_state=42)

In [14]:
train_encoded = encoder.fit_transform(train)

In [15]:
train_encoded.head()

,season_4.0,season_2.0,season_1.0,season_3.0,yr_1.0,yr_0.0,mnth_11.0,mnth_10.0,mnth_9.0,mnth_6.0,...,workingday_1.0,workingday_0.0,weathersit_2.0,weathersit_1.0,weathersit_3.0,temp,atemp,hum,windspeed,cnt
688,1,0,0,0,1,0,1,0,0,0,...,1,0,1,0,0,0.380833,0.375621,0.623333,0.235067,5499
649,1,0,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0.435000,0.431167,0.463333,0.181596,7570
637,1,0,0,0,1,0,0,0,1,0,...,0,1,0,1,0,0.542500,0.530296,0.542917,0.227604,8555
525,0,1,0,0,1,0,0,0,0,1,...,0,1,0,1,0,0.710833,0.648367,0.437083,0.144287,7498
367,0,0,1,0,1,0,0,0,0,0,...,1,0,0,1,0,0.150000,0.126275,0.441250,0.365671,2236


In [16]:
X = df.drop('cnt', axis = 1)
y = df['cnt']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [18]:
from sklearn.linear_model import LinearRegression

In [19]:
LR = LinearRegression()

In [20]:
LR.fit(X_train, y_train)

LinearRegression()

In [21]:
pred = LR.predict(X_train)

In [22]:
from sklearn.metrics import mean_squared_error,r2_score

In [23]:
mean_squared_error(y_train, pred)

767408.9969051768

In [24]:
r2_score(y_train, pred)

0.7891098322665724

In [25]:
y_pred = LR.predict(X_test)

In [26]:
mean_squared_error(y_test, y_pred)

751126.2310636705

In [27]:
r2_score(y_test,y_pred)

0.811225776607647

In [28]:
LR.coef_

array([  532.81457002,  1981.84189358,   -45.73568804,  -386.37928782,
          68.41529588,   161.51159788,  -598.60038909, -2463.45125556,
        8751.38948003,  -965.27937163, -1956.16024225])

**Year & Holiday** these are the features which have significant contribution. **The model is perfectly fitting the dataset.**